In [34]:
import pandas as pd
import os
import shutil

In [35]:
cohen_dir = 'covid-chestxray-dataset'
cohen_csv_path = f'./{cohen_dir}/metadata.csv'
cohen_img_path = f'./{cohen_dir}/images'

In [36]:
fig1_dir = 'Figure1-COVID-chestxray-dataset'
fig1_csv_path = f'./{fig1_dir}/metadata.csv'
fig1_img_path = f'./{fig1_dir}/images'

In [37]:
actmed_dir = 'Actualmed-COVID-chestxray-dataset'
actmed_csv_path = f'./{actmed_dir}/metadata.csv'
actmed_img_path = f'./{actmed_dir}/images'

In [38]:
sirm_dir = 'COVID-19-Radiography-Database'
sirm_csv_path = f'./{sirm_dir}/COVID'
sirm_img_path = f'./{sirm_dir}.xlsx'

In [39]:
rsna_csv1_path = './stage_2_detailed_class_info.csv'
rsna_csv2_path = './stage_2_train_labels.csv'
rsna_img_path = './stage_2_train_images'

In [40]:
dataset_dir = 'dataset'

images_dir = f'{dataset_dir}/images'

if not os.path.exists(dataset_dir):
    os.mkdir(dataset_dir)
    print("Dataset directory created.")

if not os.path.exists(images_dir):
    os.mkdir(images_dir)
    print("Images directory created.")

In [49]:
open(f'./{dataset_dir}/data_file.csv', 'w').close()

In [50]:
cohen_csv = pd.read_csv(cohen_csv_path, nrows=None)
print(cohen_csv.shape)

data_file = open(f'./{dataset_dir}/data_file.csv', 'w')
data_file.write('filename,diagnosis\n')

covid_positives = 0
covid_negatives = 0

for i, row in cohen_csv.iterrows():
    if row['folder'] == 'volumes':
        continue
    
    image = row['filename']
    
    src_path = f'{cohen_img_path}/{image}'
    dst_path = f'{images_dir}/{image}'
    
    shutil.copy2(src_path, dst_path)
    
    finding = row['finding'].split('/')[-1]
    
    if finding == 'COVID-19':
        sample = str(image) + ',' + 'positive' + '\n'
        covid_positives += 1
    else:
        sample = str(image) + ',' + 'negative' + '\n'
        covid_negatives += 1
    data_file.write(sample)

print('Covid positives:', covid_positives)
print('Covid negatives:', covid_negatives)

data_file.close()

(950, 30)
Covid positives: 563
Covid negatives: 366
